# **디스플레이 생산라인 센서데이터 이상치 탐지 프로젝트
## 🚀 전처리 수행 소스코드

In [1]:
# 패키지 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
# 경고 끄기
warnings.filterwarnings(action='ignore')

In [2]:
# 데이터 불러오기
# 첫번째 열을 인덱스로 사용함
fact_data = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949', index_col=0)

In [3]:
# 데이터 확인
fact_data.info()
fact_data.head()
# 840 컬럼 x 8145개 행이 있음
# 데이터 타입은 float과 int형으로 모두 숫자 데이터. 연산 가능함.

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 840 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to R.Vac
dtypes: float64(834), int64(6)
memory usage: 52.3+ MB


,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,X1WOS45105_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
dat...date.name.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
2016-01-01 10,-1.772682,-1.713219,-

## 📌1. 컬럼 제외하기

### 1_1.레이블 제외하기

In [4]:
# 레이블 확인
print(fact_data.columns[-14:])

Index(['투입.LEFT.', '투입.RIGHT.', 'ROLLDOWN.폐기.LEFT.', '진공패드.폐기.LEFT.',
       'ROLLDOWN.폐기.RIGHT.', '진공패드.폐기.RIGHT.', '폐기율...',
       'ROLLDOWN.진공패드.폐기율..LEFT.', 'ROLLDOWN.진공패드.폐기율..RIGHT.',
       'ROLLDOWN.진공패드.폐기율..전체.', 'L.RD', 'L.Vac', 'R.RD', 'R.Vac'],
      dtype='object')


In [5]:
# 레이블 제외하기
col_notfeature = fact_data.columns[-14:]

col_use = []
for col in fact_data.columns:
    if col not in col_notfeature:
       col_use.append(col)

# 개수 확인
print(len(col_use)) # 826개

# 데이터프레임에 반영
fact_data = fact_data[col_use]

826


### 1_2.Right 컬럼 제외하기
- 본 데이터의 컬럼 이름의 규칙성과 데이터의 상관관계를 살펴본 결과, left, right 공정은 동일한 공정으로 추정됨
- 동일한 공정의 데이터를 이용하는 것은 최종 모델에 특정 공정의 영향력이 중복되어 반영되는 것과 같음
- 모델링에 중복하여 영향을 미치면 중요 피처를 찾을 때 큰 영향력을 가진 것으로 판단하는 잘못을 야기할 수 있으므로, 어느 한 쪽 공정의 데이터를 제외하는 것이 타당함
- 상세한 판단 근거는 [src/1. 데이터 탐색 및 전처리/(1) 컬럼명 구분.ipynb](https://github.com/ammobam/Display_SensorData/blob/main/src/1.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%83%90%EC%83%89%20%EB%B0%8F%20%EC%A0%84%EC%B2%98%EB%A6%AC/(1)%20%EC%BB%AC%EB%9F%BC%EB%AA%85%20%EA%B5%AC%EB%B6%84.ipynb)에서 확인할 수 있음

In [6]:
# 컬럼명을 기준으로 공정을 분류해둔 딕셔너리 불러오기
with open('/content/drive/MyDrive/나무플래닛/3. 산출물/dict_left_right.pickle', 'rb') as f:
    dict_l_r = pickle.load(f)

# 확인
print(dict_l_r.keys())

# Right 공정에 해당하는 컬럼
col_right = dict_l_r['Right']
print(len(col_right)) # 91개

dict_keys(['Left', 'Right', 'Center'])
91


In [7]:
# Right 컬럼 제외하기
col_use = []
for col in fact_data.columns:
    if col not in col_right:
       col_use.append(col)
print(len(col_use)) # 739개

# 데이터프레임 만들기
fact_data = fact_data[col_use]

739


## 📌2. 결측치 처리

### 2_1. 전체 열이 결측치인 행 삭제

In [8]:
# 결측치 찾기
nan_data = fact_data.isnull().sum().sort_values(ascending=False)
print(nan_data)
## isnull() : 결측치를 찾아 True/False 리턴함.
    # boolean 데이터타입은 숫자로 변환하면 True -> 1, False -> 0
    # 여기서는 결측치가 있는 데이터는 1의 값을 갖게 됨
## sum() : 컬럼별로 값을 모두 더함
## sort_values() : 값을 크기별로 정렬함
    # ascending=False 옵션을 주면 내림차순으로 정렬함

LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
DCS_Offset_RPM_M14.1WOS45114_PV         8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
DCS_Offset_RPM_M13.1WOS45113_PV         8145
                                        ... 
ROOF.HEATING.Zone.9.1JIC31109.PV.1         0
ROOF.HEATING.Zone.8.1JIC31108.PV.1         0
ROOF.HEATING.Zone.7.1JIC31107.PV.1         0
ROOF.HEATING.Zone.6.1JIC31106.PV           0
LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101          0
Length: 739, dtype: int64


In [9]:
# 이때 결측치 개수 컬럼은 시리즈 타입이므로 슬라이싱 가능함
print(type(nan_data))

# nan 개수 상위 10개를 확인함
# 조회 결과, 7개 컬럼은 전체 8145개의 행이 모두 결측치임. 데이터가 없으므로 삭제.
print(nan_data[:10])

<class 'pandas.core.series.Series'>
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
DCS_Offset_RPM_M14.1WOS45114_PV         8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
DCS_Offset_RPM_M13.1WOS45113_PV         8145
ROOF.HEATING.Zone76.1JIC31176.PV        8145
ROOF.HEATING.Zone52.1JIC31152.PV        8145
N2.TO.EA1313A.1FI33503A_PV.1               1
LEHR.DRV.MTR..M9.2.SPD.1WSI40115.1         1
LEHR.DRV.MTR..M9.1.SPD.1WSI40114.1         1
dtype: int64


In [10]:
# 전체 행이 결측치인 컬럼을 찾고 삭제하기

# 1. 전체 행이 결측치인 컬럼을 찾기
print(nan_data[nan_data.values == fact_data.shape[0]].index.tolist())
    # nan_data에서 다음 조건에 해당하는 컬럼을 찾음
        # nan_data.values : nan_data를 데이터 배열로 만들어줌
        # fact_data.shape -> (행 개수, 열 개수)
            # fact_data.shape[0]는 fact_data 행의 개수임
    # .index : 그때의 컬럼이름(여기선 인덱스)을 추출함
    # .tolist() : 데이터를 리스트 타입으로 변환함

# 2. 해당 컬럼 삭제
fact_data.drop(nan_data[nan_data.values == fact_data.shape[0]].index.tolist(), axis=1, inplace=True)
    # inplace=True : drop 수행한 결과를 fact_data에 반영할지 여부 설정

# 3. 확인
fact_data.info()

['LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV', 'ROOF.HEATING.Zone78.1JIC31178.PV', 'DCS_Offset_RPM_M14.1WOS45114_PV', 'UPPER.RET..Z218.TMP.1TIC42601.PV', 'DCS_Offset_RPM_M13.1WOS45113_PV', 'ROOF.HEATING.Zone76.1JIC31176.PV', 'ROOF.HEATING.Zone52.1JIC31152.PV']
<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 732 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to STD_GLS_L
dtypes: float64(732)
memory usage: 45.5+ MB


### 2_2. 그외 결측치 처리

In [11]:
# 그 외 결측치 처리
# 남은 결측치는 ffill로 채움
# 이 데이터는 1시간마다 측정한 센서 데이터이므로 전체 컬럼에 대한 평균보다는 이전 행의 값으로 결측치를 처리하는 것이 적절함
fact_data.fillna(method = 'ffill', inplace=True)
    # inplace=True : fillna 수행한 결과를 fact_data에 반영할지 여부 설정

# 확인
fact_data.isnull().sum().sort_values()

LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101     0
ROOF.HEATING.Zone.4.1JIC31104.PV.1    0
ROOF.HEATING.Zone.5.1JIC31105.PV      0
ROOF.HEATING.Zone.6.1JIC31106.PV      0
ROOF.HEATING.Zone.7.1JIC31107.PV.1    0
                                     ..
LOWER.C3..Z193TMP.1TIC42454.PV        0
LOWER.C3..Z196.TMP.1TIC42457.PV       0
UPPER.C4..Z197.TMP.1TIC42501.PV       0
UPPER.C4..Z199.TMP.1TIC42503.PV       0
STD_GLS_L                             0
Length: 732, dtype: int64

## 📌3.데이터 전처리

### 3_1. 분산 0인 데이터 제거
- 분산이 0인 데이터를 제거하는 이유?
    - 어떤 피처의 분산이 0이라는 것은 그 피처의 데이터가 모든 행에 대해 거의 변하지 않은 것을 의미함
    - 어떤 경우에도 같은 값을 내는 컬럼이 불량률에 영향을 주고 있다고 보기 어려움

In [12]:
# 분산 구하기
df_var = fact_data.var()    # 시리즈
# 확인
print("분산 통계량 확인\n", df_var.describe())

# 분산이 0에 수렴하는 컬럼 확인
## 10e-10 이하면 0에 수렴하는 값으로 간주함
print(df_var[df_var.values <= 10e-10])

# 삭제
fact_data.drop(df_var[df_var.values <= 10e-10].index, axis=1, inplace=True)

# 컬럼 수 확인
print("컬럼 수:", len(fact_data.columns))

분산 통계량 확인
 count    7.320000e+02
mean     9.906385e-01
std      9.741771e-02
min      7.538541e-27
25%      1.000222e+00
50%      1.000674e+00
75%      1.000815e+00
max      1.001149e+00
dtype: float64
LEHR.DRV.MTR..M4.2.CUR.1WII40105.PV     7.538541e-27
LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV     7.538541e-27
LEHR.DRV.MTR..M9.2.CUR.1WII40115.PV     7.538541e-27
LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV    7.538541e-27
LEHR.DRV.MTR..M14.1.CUR.1WII40124.PV    7.538541e-27
LEHR.DRV.MTR..M17.2.CUR.1WII40131.PV    7.538541e-27
TMP.TIN..BAY.4.LEFT.1TI30203.PV         7.538541e-27
dtype: float64
컬럼 수: 725


### 3_2.피처 간 상관관계 높은 피처 제거
- 상관관계 : 피처 간의 종속된 정도
- 상관관계가 높은 피처를 제거하는 이유?
    - 두 피처 간의 상관관계가 높다는 것은 하나의 피처 값이 다른 피처의 값에 큰 영향을 주고있음을 뜻함
    - 두 피처는 동일한 원인에 기인하여 변하는 것으로 추측할 수 있음
    - 이를 제거하지 않고 두면 같은 원인에서 발생한 데이터가 모델링에 여러 번 반영됨
    - 모델링에 영향을 미치는 원인들이 모두 비슷한 중요도로 반영되게 하려면 종속성이 낮은 피처들만을 이용하여 모델을 만드는 것이 타당함
- 여기서는 피처 간 상관계수의 절대값이 0.9 이상인 경우를 종속된 것으로 봄
- 상세한 소스 설명은 [src/1. 데이터 탐색 및 전처리/(3) 상관계수 높은 피처 제거.ipynb]()에서 확인할 수 있음


In [13]:
# 모든 컬럼에 대해 상관계수를 구하고 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거

# 모든 컬럼에 대해 상관계수의 절대값 행렬 데이터프레임 구하기
corr_arr = abs(fact_data.corr())

# 삭제할 컬럼 저장 리스트 생성
del_list = []

for i, col in enumerate(corr_arr.columns):
    for j in range(len(corr_arr.index)):
        # 동일 컬럼의 경우에는 상관계수가 1이므로 넘어가도록 함
        if i == j:
            continue
        # 이미 del_list에 있는 경우는 다음
        elif corr_arr.index[i] in del_list:           
            break
        # del_list에 포함되지 않고 상관계수가 0.9 이상인 조합에 대해 한 쪽의 컬럼이름 저장
        if np.abs(corr_arr.iloc[j,i]) >= 0.9:
            del_list.append(corr_arr.index[j])
# 확인
print(len(del_list))

630


In [14]:
# del_list에 있는 컬럼은 제외
total_col = fact_data.columns.tolist()
col_use = []
for c in total_col:
    if c not in del_list:
        col_use.append(c)
print("컬럼 수", len(col_use))

# 데이터프레임에 반영
fact_data = fact_data[col_use]

컬럼 수 324


## 📌4.전처리 수행 후 파일 저장

In [15]:
# 전처리 수행한 파일 저장하기
fact_data.to_csv('/content/drive/MyDrive/나무플래닛/3. 산출물/fact_data.csv')

In [16]:
# 확인
fact_data= pd.read_csv('/content/drive/MyDrive/나무플래닛/3. 산출물/fact_data.csv')
fact_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145 entries, 0 to 8144
Columns: 325 entries, dat...date.name. to STD_GLS_L
dtypes: float64(324), object(1)
memory usage: 20.2+ MB
